## 詞頻矩陣

In [9]:
import jieba
jieba.load_userdict('userdict.txt')
ary = ['富邦人壽首創保費1萬就能刷卡',
       '富邦人壽不老勇士Go來盃3對3籃賽南北開打',
       '富邦人壽不老勇士Go來盃開打']

In [10]:
corpus = []
for title in ary:
    corpus.append(' '.join(jieba.cut(title)))

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() 

X = vectorizer.fit_transform(corpus)

In [12]:
X.shape

(3, 11)

In [13]:
vectorizer.get_feature_names()

['go', '不老', '人壽', '保費', '刷卡', '勇士', '南北', '富邦', '籃賽', '開打', '首創']

In [14]:
X.toarray()

array([[0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]], dtype=int64)

## 相似度計算

In [17]:
a = '柯P 大巨蛋 遠雄'
b = '柯P 為了 大巨蛋 一案 找 遠雄 算帳'

#   柯P 大巨蛋 遠雄 為了 一案 找 算帳
# a  1    1    1    0    0   0   0
# b  1    1    1    1    1   1   1

In [25]:
import numpy as np
na = np.array([1,1,1,0,0,0,0])
nb = np.array([1,1,1,1,1,1,1])

np.sqrt(sum((nb - na) ** 2))

2.0

In [27]:
sum(na * nb) / (np.sqrt(sum(na ** 2)) * np.sqrt(sum(nb ** 2)) )

0.6546536707079772

In [23]:
a = '柯P 大巨蛋 遠雄'
b = '柯P 為了 大巨蛋 一案 找 遠雄 算帳 因為 遠雄 在 這案 中 出爾反爾'

#   柯P 大巨蛋 遠雄 為了 一案 找 算帳 因為 在 這案 中 出爾反爾
# a  1    1    1    0    0   0   0  0   0   0   0    0
# b  1    1    2    1    1   1   1  1   1   1   1    1

In [28]:
import numpy as np
na = np.array([1,1,1,0,0,0,0,0,0,0,0,0,0])
nb = np.array([1,1,2,1,1,1,1,1,1,1,1,1,1])

np.sqrt(sum((nb - na) ** 2))

3.3166247903553998

In [29]:
sum(na * nb) / (np.sqrt(sum(na ** 2)) * np.sqrt(sum(nb ** 2)) )

0.57735026918962584

In [31]:
import jieba
jieba.load_userdict('userdict.txt')
ary = ['富邦人壽首創保費1萬就能刷卡',
       '富邦人壽不老勇士Go來盃3對3籃賽南北開打',
       '富邦人壽不老勇士Go來盃開打']
corpus = []
for title in ary:
    corpus.append(' '.join(jieba.cut(title)))
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() 

X = vectorizer.fit_transform(corpus)
X.toarray()

array([[0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]], dtype=int64)

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(X)

array([[ 1.        ,  0.31622777,  0.4       ],
       [ 0.31622777,  1.        ,  0.79056942],
       [ 0.4       ,  0.79056942,  1.        ]])

In [33]:
from sklearn.metrics.pairwise import cosine_distances
cosine_distances(X)

array([[ 0.        ,  0.68377223,  0.6       ],
       [ 0.68377223,  0.        ,  0.20943058],
       [ 0.6       ,  0.20943058,  0.        ]])

## 新聞推薦引擎

In [45]:
import requests
from bs4 import BeautifulSoup
news_ary = []
for page in range(1,30):
    res = requests.get('https://tw.appledaily.com/new/realtime/{}'.format(page))
    soup = BeautifulSoup(res.text, 'html.parser')
    for rec in soup.select('.rtddt a'):
        dic = {}
        dic['link'] = rec['href']
        res2 = requests.get(dic['link'])
        soup2 = BeautifulSoup(res2.text, 'html.parser')
        dic['category'] = soup2.select('.current')[2].text
        dic['title']    = soup2.select_one('h1').text
        dic['content']  = soup2.select_one('.ndArticle_margin p').text
        news_ary.append(dic)
    print(page)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [46]:
import pandas
newsdf = pandas.DataFrame(news_ary)
newsdf.to_excel('appledaily20171214.xlsx')

In [48]:
import pandas
newsdf = pandas.read_excel('data/appledaily20171214.xlsx')

In [49]:
newsdf.head()

,category,content,link,title
0,政治,新增：立委說法民進黨立法院黨團預計在明天的院會中，讓改制農田水利會的《農田水利會組織通則》修...,https://tw.news.appledaily.com/politics/realti...,【更新】水利會改官派明闖關 綠委24小時前顧議場大門防藍突襲
1,論壇,邱俊棠／台中市民、見習醫師；曾任台灣醫學生聯合會對外副會長對於公民參與公眾事務而能得到單位首...,https://tw.news.appledaily.com/forum/realtime/...,請中市府為所當為 加速中火燃煤限制
2,社會,被控來台涉發展情報組織的中國學生周泓旭，因接觸我方外交部官員而露餡落網，今年9月被台北地院一...,https://tw.news.appledaily.com/local/realtime/...,陸生共諜嗆台司法 「不敢公開審理我」
3,生活,【廣編特輯】 對於全能神教會在中國所遭受的迫害與在韓國、香港及臺灣所面對的反面宣傳，非政府組...,https://tw.news.appledaily.com/life/realtime/2...,【特企】NGO聯名譴責中共迫害宗教信仰
4,生活,立法院交通委員會今審查《道路交通管理處罰條例》，通過營業大客車駕駛人行駛一般道路未繫安全帶，...,https://tw.news.appledaily.com/life/realtime/2...,乘客搭大客車不配合繫安全帶 立院初審通過可罰6千元


In [52]:
corpus = []
titles = []
for news in newsdf.iterrows():
    corpus.append(' '.join(jieba.cut(news[1]['content'])))
    titles.append(news[1]['title'])

In [56]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X

<899x38624 sparse matrix of type '<class 'numpy.int64'>'
	with 128131 stored elements in Compressed Sparse Row format>

In [67]:
from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(X)
cs.shape

(899, 899)

In [65]:
na = np.array([1,9,2,6,8,4,5,7])
na.sort()
na

array([1, 2, 4, 5, 6, 7, 8, 9])

In [66]:
na = np.array([1,9,2,6,8,4,5,7])
na.argsort()

array([0, 2, 5, 6, 3, 7, 4, 1], dtype=int64)

In [72]:
rank = cs[0].argsort()[::-1]
for rec in rank[0:10]:
    if cs[0][rec] > 0.3:
        print(titles[rec], cs[0][rec])

【更新】水利會改官派明闖關　綠委24小時前顧議場大門防藍突襲 1.0
反對水利會改制　吳敦義下令：藍委做好夜宿立院抗爭準備 0.367845249572
農田水利會改公務機關　蔡英文：這不是綁樁 0.327587152559


In [73]:
def getSimiliarArticle(pos):
    rank = cs[pos].argsort()[::-1]
    for rec in rank[0:10]:
        similiarities = cs[pos][rec]
        if similiarities > 0.3:
            print(titles[rec], similiarities)

In [79]:
getSimiliarArticle(6)

喬妹升格宋太太首上工　現身中國見習近平 1.0
素顏依舊完美　喬妹升格人妻赴中開工 0.31870904608


## 同義詞

In [16]:
synonym_dic = {
    '柯P': '柯文哲'
}
s = '柯P為了大巨蛋一案找遠雄算帳'
' '.join([synonym_dic.get(ele, ele) for ele in jieba.cut(s)])


'柯文哲 為 了 大巨蛋 一案 找遠 雄算帳'